# Testcase Data Retrieval for Earthquake Swarms in the PNSN Network
This notebook provides a worked example of data retrieval for the following earthquake swarm within the PNSN network:  
The May 2017 earthquake swarm near Bremerton  
        - **Note** this example uses just 2 days of the swarm: May 10-11, 2017.  
        - Catalog searches show the swarm can broadly be defined between May 8 and November 2017  
        - ...potentially longer - a declustering analysis would better define this.  
![Event map of earthquakes in the seismic swarm near Bremerton, WA](data/2017-05-10_BRM/image.png)  


- "Earthquake swarm NE of Bremerton" blog post by Renate Hartog on May 11, 2017  
https://www.pnsn.org/blog/2017/05/11/earthquake-swarm-ne-of-bremerton


### Script Author
Nathan Stevens: ntsteven@uw.edu

In [1]:
# Import dependencies
from obspy import UTCDateTime, Stream
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
import pandas as pd
import matplotlib.pyplot as plt
import os


# Define small helper function - I know it's somewhere in ObsPy, but this was easier than hunting it down...
def m2deg(x):
    """
    Convert distance `x` from meters to great-circle degrees on Earth
    """
    return x/111139.

In [2]:
# Initialize Client
client = Client('IRIS')

### Data selection
Use the following fields to define an event and station query kwargs for use with the `obspy`
 - **starttime**: [obspy.UTCDateTime] Start time of query in UTC
 - **endtime**:      [obspy.UTCDateTime] End time of query in UTC
 - **latitude**:     [float] centroid latitude for queries in *degrees North*
 - **longitude**:    [float] centroid longitude for queries in *degrees East*
 - **maxradius**: [float] maximum radius for query in *degrees*

In [6]:
# Define fields for query
fields = ('starttime','endtime','latitude','longitude','maxradius')
TSstr = "2017-05-11T00:00:00"
TEstr = "2017-05-11T06:00:00"
# TEstr = "2017-05-12T23:59:59"
# Define data selection kwargs for events from IRIS catalog
qevent_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                UTCDateTime(TEstr),\
                                47.5828,-122.57841,m2deg(7.5e3)]))
# Define data selection kwargs for stations in IRIS catalog
qstation_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                   UTCDateTime(TEstr),\
                                47.5828,-122.57841,m2deg(20e3)]))

# Limit channels to those coming from broadbands ?H? and accelerometers ?N?
CLIST = 'HHZ,HHN,HH1,HHE,HH2,ENZ,EN1,ENN,EN2,ENE,BHZ,BHN,BHZ'

In [8]:
# Download inventory with specified query down to channel 
# (ML generally operates on data w/o instrument response correction)
inv = client.get_stations(**qstation_kwargs,level='channel',channel=CLIST)#,channel='??[ZNE12]')
print(inv)

Inventory created at 2023-10-06T15:54:50.873500Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2017-05-...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			UW
		Stations (19):
			UW.BABE (Bainbridge, WA, USA)
			UW.GMW (Gold Mountain, WA, USA)
			UW.GNW (Green Mountain, WA, USA)
			UW.GTWN (Georgetown Playfield ANSS-SMO)
			UW.HART (Harbor Island, Seattle, WA, USA)
			UW.HOLY (Holy Rosary ANSS-SMO)
			UW.KCAM (King County Airport Maintenance Shop, Seattle, WA, USA)
			UW.KDK (Kingdome, Seattle, WA, USA)
			UW.KIMR (Kitsap County HHW Collection Facility, Bremerton, WA, USA)
			UW.KINR (North Road Shop, Poulsbo, WA, USA)
			UW.KITP (Kitsap Treatment Plant, WA, USA)
			UW.LAWT (Lawton, Seattle, WA, USA)
			UW.MNWA (Manchester, WA)
			UW.PSNS (Puget Sound Naval Shipyard - ANSS SMO)
			UW.RAD1 (RAD site 1, Seattle, WA)
			UW.RAD4 (RAD site 4, Seattle, WA)
			UW.RAD6 (RAD site 6, Seattle, 

In [ ]:
# # Use IRIS webservices / FDSN to query waveform data 
# # Based on mass_downloader example on docs.github.org: https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html
# domain = CircularDomain(**dict((k_,qstation_kwargs[k_]) for k_ in ['latitude','longitude','maxradius']),minradius=.0)

# restrictions = Restrictions(starttime=UTCDateTime(TSstr),endtime=UTCDateTime(TEstr),
#                             chunklength_in_sec=3600.)
#                             # limit_stations_to_inventory=inv,chunklength_in_sec=3600)
# mdl = MassDownloader()
# mdl.download(domain,restrictions,threads_per_client=2,\
#              mseed_storage=os.path.join('data','2017-05_BRM'),\
#              stationxml_storage=os.path.join('data','2017-05_BRM','{network}','{station}.xml'))

In [20]:
inv.networks[0].stations[10].available_channels

AttributeError: 'Station' object has no attribute 'available_channels'

In [23]:
# Get continuous waveforms using BULK request - this successfully pulls all data
ts = UTCDateTime(TSstr)
te = ts + 3600.
out_file_FMT = os.path.join('data','2017-05_BRM','{NET}','{STA}','{NET}.{STA}.{CHANS}.{TS}.mseed')
bulk = []         
while ts < UTCDateTime(TEstr):
    print('Running window %s -- %s'%(ts.isoformat(),te.isoformat()))
    # Compose bulk request
    bulk = []
    for N_ in inv.networks:
        for S_ in N_.stations:
            try:
                os.makedirs(os.path.join('data','2017-05_BRM',N_.code,S_.code))
            except:
                pass
            for C_ in S_.channels:
                CC_ = C_.code
                LC_ = C_.location_code
                bulk.append((N_.code,S_.code,LC_,CC_,ts,te))
    # Execute bulk request
    st = client.get_waveforms_bulk(bulk)
    # Save waveforms in ML-compliant ordering - MSEED @ station level with Z,[N1],[E2] channel sequencing
    for N_ in inv.networks:
        for S_ in N_.stations:
            sst = st.copy().select(network=N_.code,station=S_.code)
            nchans = len(sst)

            # If 3-C station, bundle data as normal
            if nchans == 3:
                ist = Stream()
                ist += sst.select(channel='??Z')
                ist += sst.select(channel='??[N1]')
                ist += sst.select(channel='??[E1]')            
                CNAMES = ist[0].stats.channel + ist[1].stats.channel[-1] + ist[2].stats.channel[-1]
                outname = out_file_FMT.format(NET=N_.code,STA=S_.code,CHANS=CNAMES,TS=ts.isoformat())
                ist.write(outname,fmt='MSEED')
            # If 1-C station, use the "trick" from Retailleau et al. (2022)
            elif nchans == 1:
                ist = Stream()
                ist += sst[0]
                tr1f = sst[0].copy()
                tr1f.stats.channel+= 'N'  
                ist += tr1f
                tr2f = sst[0].copy()
                tr2f.stats.channel+= 'E'
                ist += tr2f
                CNAMES = sst[0].stats.channel + 'ZZ'
                outname = out_file_FMT.format(NET=N_.code,STA=S_.code,CHANS=CNAMES,TS=ts.isoformat())
                ist.write(outname,fmt='MSEED')
            # Deal with multi-sensor/-sampling_rate stations
            elif nchans > 3 and nchans%3 == 0:
                CPlist = []
                for C_ in S_.channels:
                    if C_.code[:2] not in CPlist:
                        CPlist.append(C_.code[:2])
                for CP_ in CPlist:
                    ist = Stream()
                    ist += sst.select(channel=CP_+'Z')
                    ist += sst.select(channel=CP_+'[N1]')
                    ist += sst.select(channel=CP_+'[E1]')            
                    CNAMES = ist[0].stats.channel + ist[1].stats.channel[-1] + ist[2].stats.channel[-1]
                    outname = out_file_FMT.format(NET=N_.code,STA=S_.code,CHANS=CNAMES,TS=ts.isoformat())
                    ist.write(outname,fmt='MSEED')
            
    # Advance indices for WHILE loop
    ts += 3600.
    te += 3600.

Running window 2017-05-11T00:00:00 -- 2017-05-11T01:00:00
Running window 2017-05-11T01:00:00 -- 2017-05-11T02:00:00
Running window 2017-05-11T02:00:00 -- 2017-05-11T03:00:00
Running window 2017-05-11T03:00:00 -- 2017-05-11T04:00:00
Running window 2017-05-11T04:00:00 -- 2017-05-11T05:00:00
Running window 2017-05-11T05:00:00 -- 2017-05-11T06:00:00


In [5]:
# Download catalog from Client
cat = client.get_events(**qevent_kwargs)
print(cat.__str__(print_all=True))

1 Event(s) in Catalog:
2017-05-11T00:41:03.700000Z | +47.598, -122.546 | 0.98 Ml


In [7]:
# Load PNSN queried catalog data from custom queries for Bremerton-Adjacent Swarm (BRM)
df_BRM_full = pd.read_csv(os.path.join('data','2017-05_BRM','pnsn_event_catalog_2017_BRM_cluster.csv'),\
                          parse_dates=['Time UTC'],index_col='Time UTC')
# Sort by index
df_BRM_full = df_BRM_full.sort_index()
# Filter by query times
# Translate from obspy.UTCDateTime --> pandas.Timestamp
pdTS = pd.Timestamp(TSstr)
pdTE = pd.Timestamp(TEstr)

# Filter to period of interest
df_EVE = df_BRM_full[(df_BRM_full.index >= pdTS) & (df_BRM_full.index <= pdTE)]
print(len(df_EVE))
print(df_EVE)


18
                         Evid  Magnitude Magnitude Type    Epoch(UTC)  \
Time UTC                                                                
2017-05-11 00:41:03  61264902        1.0              l  1.494463e+09   
2017-05-11 01:49:33  61264912        0.3              d  1.494467e+09   
2017-05-11 02:31:17  61264922        1.5              l  1.494470e+09   
2017-05-11 07:35:24  61265017        3.5              l  1.494488e+09   
2017-05-11 07:40:44  61265022        1.0              l  1.494488e+09   
2017-05-11 07:41:13  60134708        0.3              d  1.494488e+09   
2017-05-11 08:19:37  61265037        1.3              l  1.494491e+09   
2017-05-11 08:37:57  61265047        1.5              l  1.494492e+09   
2017-05-11 09:00:56  61265062        1.1              l  1.494493e+09   
2017-05-11 09:04:43  61265067        0.8              l  1.494493e+09   
2017-05-11 09:19:09  61265072        2.1              l  1.494494e+09   
2017-05-11 09:27:40  61265077        1.1        

In [ ]:
# # Use inventory to pull data for the period of interest and save to disk in 1 hour chunks
# ts = UTCDateTime(TSstr)
# te = ts + 3600.
# save_dir = os.path.join('data','2017-05_BRM')
# while ts < UTCDateTime(TEstr):
#     st = client.get_waveforms(starttime=ts,endtime=te)
#     ts += 3600.
#     te += 3600.


In [28]:
# # Get continuous waveforms - OBSOLITED - This method fails to get extensive data
# ts = UTCDateTime(TSstr)
# te = ts + 3600.
# out_file_FMT = os.path.join('data','2017-05_BRM','{NET}','{STA}','{NET}.{STA}.{LOC}.{CHAN}.{TS}.mseed')
             
# while ts < UTCDateTime(TEstr):
#     print('Running window %s -- %s'%(ts.isoformat(),te.isoformat()))
#     for N_ in inv.networks:
#         # try:
#         #     os.makedirs(os.path.join('data','2017-05_BRM',str(N_.code)))
#         # except:
#         #     pass
#         for S_ in N_.stations:
#             st = Stream()
#             # try:
#             #     os.mkdir(os.path.join('data','2017-05_BRM',str(N_.code),str(S_.code)))
#             # except:
#             #     pass
#             for C_ in S_.channels:
#                 CC_ = C_.code
#                 LC_ = C_.location_code
#                 out_file = out_file_FMT.format(NET=N_.code,STA=S_.code,LOC=LC_,CHAN=CC_,TS=ts.isoformat())
#                 try:
#                     tr = client.get_waveforms(N_.code,S_.code,LC_,CC_,ts,te)[0]
#                     try: 
#                         os.mkdirs(os.path.join('data','2017-05_BRM',str(N_.code),str(S_.code)))
#                     except:
#                         pass
#                     tr.write(out_file,fmt='MSEED')
#                 except:
#                     pass
#     ts += 3600.
#     te += 3600.

Running window 2017-05-11T00:00:00 -- 2017-05-11T01:00:00
Running window 2017-05-11T01:00:00 -- 2017-05-11T02:00:00
Running window 2017-05-11T02:00:00 -- 2017-05-11T03:00:00


In [ ]:
# # Plot the catalog
# fig = plt.figure(figsize=(10,10))
# axa = fig.add_subplot(211)
# axa.plot(df_BRM_full['Magnitude'],'.')
# axa.set_xlabel('UTC Date Time')
# axa.set_ylabel('Magnitude (mixed methods)')
# axb = axa.twinx()
# axb.plot(df_BRM_full['Magnitude'].cumsum(),'r-',alpha=0.5)
# axb.set_ylabel('Cumulative magnitude',labelpad=15)

# # # Define window to assess
# # TS = pd.Timestamp("2017-05-11")
# # TE = pd.Timestamp("2017-05-13T00:00:00")

# axa.fill_between([TS,TE],[axa.get_ylim()[0]]*2,[axa.get_ylim()[1]]*2,color='yellow')


# axc = fig.add_subplot(212)
# IND = (df_BRM_full.index >= TS) & (df_BRM_full.index <= TE)
# axc.plot(df_BRM_full[IND]['Magnitude'],'.')
# axd = axc.twinx()
# axd.plot((10**df_BRM_full[IND]['Magnitude']).cumsum(),'r-',alpha=0.5)

# # axc = fig.add_subplot(223)
# # ch = axc.scatter(df_BRM_full['Lon'],df_BRM_full['Lat'],s = 5**df_BRM_full['Magnitude'],c=df_BRM_full.index)
# # plt.colorbar(ax=axc,cax=ch)